# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal


### Projet A2023

-----

# Prix des diamants

### Contexte

Dans le cadre du projet du cours MTH3302 on réalise une Prédiction du prix de vente des diamants. Cet ensemble classique de données classique contient les prix les caractéristiques de près de 54 000 diamants. Il s'agit d'un ensemble de données idéal pour mettre en pratique les modèles statistiques vus en MTH3302 !

### Objectif

Notre objectif est de construire un modèle prédictif pour prédire les prix des diamants selon leurs propriétés physiques, tels que leur qualité de coupe, leur couleur, leur clarité, ainsi que leurs dimensions.

### Données

Les données sont constituées des fichiers suivants :

- `train.csv`
- `test.csv`

On utilise le fichier train.csv qui contient le prix de vente en dollar américain de 40 455 diamants en fonction des caractéristiques suivantes :

- `cut` : qualité de coupe (Fair, Good, Very Good, Premium, Ideal)
- `color` : couleur du diamant (de J (pire) à D (meilleure)
- `clarity` : clarté du diamant (I1 (pire), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (meilleure))
- `x`: longueur en mm
- `y`: largeur en mm
- `z`: profondeur en mm
- `depth`: pourcentage de la profondeur exprimée comme 2*z/(x+y)
- `table`: pourcentage de la largeur du sommet du diamant par rapport au point le plus large

Le fichier test.csv contient les caractéristiques de 13 485 diamants pour lesquels vous devriez prédire le prix de vente.

## Chargement des données

Importation des librairies utilisées dans le calepin.

In [ ]:
using CSV, DataFrames, Random, MLBase, Gadfly, GLM, Statistics, Distributions, StatsBase, StatsModels

Selection des données

In [ ]:
data_dir = "../data"
data_train_filename = joinpath(data_dir, "train.csv")
data = CSV.read(data_train_filename, DataFrame)
first(data, 5)

#### Données des diamants

In [ ]:
data_test_filename = joinpath(data_dir, "test.csv")
test = CSV.read(data_test_filename, DataFrame)
first(test, 5)

#### Données du test

## Analyse Exploratoire

Pour commencer, nous allons nous familiariser avec les données afin de voir comment chaque propriété affecte le prix et de chercher les données manquantes ainsi que détecter les données abérantes.

### Description des variables explicatives

In [ ]:
describe(data)

Dans le tableau ci-dessus, nous permet de constater que:

- Le minimum de la variable de mesure x est zéro.
- Le minimum de la variable de mesure y est -0.02.
- Le minimum de la variable de mesure z est zéro.

"x","y" et "z" doivent être des valeurs plus grande que zéro, car elles définissent des mesures. Puisqu'elles sont fausses, il faut les filtrer de sorte à garder que les mesures positives. On remarque aussi qu'il y a 2200 valeurs de mesures "y" et de "depth" manquantes. On verra comment elles seront traiter plus loin dans le rapport.

### Prix en fonction de x, y et z

In [ ]:
Gadfly.set_default_plot_size(27.5cm, 10cm)
hstack(
    plot(data, x=:x, y=:price, Geom.point, Guide.title("Prix en fonction de x")),
    plot(data, x=:y, y=:price, Geom.point, Guide.title("Prix en fonction de y")),
    plot(data, x=:z, y=:price, Geom.point, Guide.title("Prix en fonction de z")),
)


Ces graphiques permettent de déceler une certaine corrélation positive entre les mesures de x, y et z et le prix du diamant. Cette corrélation semble particulièrement forte. Ceci est logique, car la mesure du diamant a un impact important sur le prix. Cependant, il y a quelques valeurs extrêmes qu'on doit traiter.

### Prix en fonction du depth et de table

In [ ]:
Gadfly.set_default_plot_size(27.5cm, 10cm)
hstack(
    plot( dropmissing(data, :depth), y=:price, x=:depth, Geom.histogram, Guide.title("Prix en fonction de depth")),
    plot(data, y=:price, x=:table, Geom.histogram, Guide.title("Prix en fonction de table")),
)

### Prix en fontion de la qualité de coupe, de la couleur du diamant et de la clarté du dianmant

In [ ]:
Gadfly.set_default_plot_size(27.5cm, 30cm)
sort_color = sort(data, :color)
vstack(
    plot(dropmissing(data[:, [:cut, :price]]), y=:price, x=:cut, Geom.boxplot, Guide.title("Prix en fonction de la qualité de coupe")),
    plot(sort_color, y=:price, x=:color, Geom.boxplot, Guide.title("Prix en fonction de la couleur")),
    plot(data, y=:price, x=:clarity, Geom.boxplot, Guide.title("Prix en fonction de la clarté")),
)

Malgré quelques valeurs abérantes, on remarque une distribution normale du prix dans les deux cas. 
On remarque que la courbe donnant le prix en fonction de $depth$ est légèrement étendue vers la gauche, tandis que celle donnant le prix en fonction de $table$ est légèrement étendue vers la droite. Cependant ce déséquilibre est très léger.

De manière générale, les variables $depth$ et $table$ ne semblent pas avoir une grande corrélation avec le prix d'un diamant.

### Largeur (y) en fonction de la longueur (x)

Ces deux valeurs étant des mesures des dimensions d'un diamant pourraient avoir une corrélation qui aiderait à prédire les valeurs de largeur manquantes. Regardons le graphique suivant.

In [ ]:
plot(data, x=:x, y=:y, Geom.point, Guide.title("Largeur en fonction de la Longueur"))

En regardant le graphique ci-dessus, nous pouvons voir une forte relation linéaire entre les valeurs de x et y. Ainsi, nous pouvons appliquer une régression linéaire simple afin de prédire les valeurs manquantes de y.

## Traitement des données abérantes

On cherche ici à écarter les valeurs abérantes, c'est-à-dire les valeurs qui sont isolées et extrêmement loin des autres valeurs. Ces données pourraient avoir une forte influence et potentiellement fausser nos prédictions.  

Pour cela, on sélectionne les valeurs qui sont à une distance de 1.5 fois l'écart interquartile, soit avant le 1er quartile, soit après le 3e quartile. Notre fonction $upperLowerTrail()$ nous permet de trouver les bornes de notre intervalle acceptant. Nous allons ensuite remplacer les données aberrantes par les limites de l'intervalle de confiance afin de maximiser la quantité de données à notre disposition à l'aide de la fonction $replaceExtremeValues()$.

In [ ]:
# Calculates the upper and lower tails of a column
function upperLowerTrail(data::DataFrame, column::Symbol)
    IQR = iqr(data[!, column])
    q1 = percentile(data[!, column], 25)
   
    q3 = percentile(data[!, column], 75)
    upper_tail = round(q3 + IQR *1.5, digits = 2)
    lower_tail = round(q1 - IQR *1.5, digits = 2)
    
    println("INF : " ,lower_tail)
    println("SUP : " ,upper_tail)

    return upper_tail, lower_tail
end

In [ ]:
function replaceExtremeValues(dataFrame::DataFrame, column::Symbol)
    upper_tail, lower_tail = upperLowerTrail(dataFrame, column)
    col = dataFrame[!, column]
    col = map(x -> if x > upper_tail upper_tail elseif x < lower_tail lower_tail else x end, col)
    dataFrame[!, column] = col
    return dataFrame
end

### Données abérantes de x

Traitons maintenant les valeurs abérantes de x. En trouvant l'intervalle de confiance, nous trouvons que la borne inférieure est de 1.96 et la borne supérieure de 9.28. Nous allons remplacer les données abérantes par celles-ci.

In [ ]:
replaceExtremeValues(data, :x)

### Données abérantes de y

Pour pouvoir calculer l'intervalle de confiance pour les valeurs de y, nous devons enlever les données manquantes. Cependant, nous allons dupliquer l'ensemble d'entraînement et retirer les valeurs manquantes afin de ne pas perdre les lignes dans celle-ci.  

In [ ]:
data_clone = data[.!ismissing.(data.y), :]

In [ ]:
replaceExtremeValues(data_clone, :y)
describe(data_clone)

### Données abérantes de z

Nous avons 22 données de y abérantes. On va donc les retirer.

In [ ]:
replaceExtremeValues(data, :z)

### Données abérantes de depth

Vue qu'il y a beaucoup de donnés et que le depth est un pourcentage qui est calculé à partir des valeurs de mesures x,y et z. Nous ne pouvons pas les enelver. Cependant on peut trouver un moyen pour corriger certaines données qui sont fausses grâce à l'équation.



### Données abérantes de table

Les données de la variables tables sont des rapports, il faut voir comment elles agissent sur le prix. On va tester avec et sans ces valeurs pour déterminer la meilleure approche.

In [ ]:
replaceExtremeValues(data, :table)

## Traitement des données manquantes

Il y a des données manquantes pour les variables y et depth. La variable depth dépend des données de y, car elle est calculée à partir de l'équation, 2*z/(x+y). Ainsi, en corrigeant y, on peut recalculer le depth avec l'équation. Grâce à la linéarité entre x et y, nous allons appliquer une régression linéaire simple afin de déduire les valeurs de y manquantes. La création de ce modèle de régression linéaire simple se trouve dans la fonction $predictMissingY()$

In [ ]:
function predictMissingY(dataframe::DataFrame, dataToChange::DataFrame)
    # Extraction des données dans des vecteurs
    x₁ = dataframe[!, :x]
    y = dataframe[!, :y]

    # Calcul des statistiques utiles
    n = length(y)
    x̄₁ = mean(x₁)
    ȳ = mean(y)

    # Calcul des coefficients de régression (pente et ordonnée à l'origine)
    β̂₁ = sum((x₁[i] - x̄₁)*(y[i] - ȳ)  for i=1:n) / sum( (x₁[i] - x̄₁)^2 for i=1:n)
    β̂₀ = ȳ - β̂₁*x̄₁

    # Calcul des résidus
    SSE = sum((y[i] - β̂₀ - β̂₁*x₁[i])^2 for i=1:n)
    SST = sum((y[i] - ȳ)^2 for i=1:n)
    SSR = sum((y[i] - β̂₀ - β̂₁*x₁[i])^2 for i=1:n)

    # Calcul du coefficient de détermination
    R² = 1 - SSR/SST
    println("R² = ", R²)
                        
    for row in eachrow(dataToChange)
    # Check if the y value is missing
        if ismissing(row[:y])
            # Predict the y value using the estimated coefficients
            row[:y] = round(β̂₀ + β̂₁ * row[:x], digits = 2)
#             row[:depth] = round((2 * row[:z] / (abs(row[:x]) + abs(row[:y]))) * 100, digits=1 )
        end
    end
end


In [ ]:
predictMissingY(data_clone, data)
replaceExtremeValues(data, :y)

In [ ]:
describe(data)

On peut à présent utiliser les valeurs extimées de $y$ pour estimer à leur tour les valeurs manquantes de $depth$

In [ ]:
function replaceMissingDepth(dataframe::DataFrame)
    for i in 1:nrow(dataframe)
        if ismissing(dataframe[i, :depth]) || isnan(dataframe[i, :depth])
            dataframe[i, :depth] = round(2 * (dataframe[i, :z] / (dataframe[i, :x]+dataframe[i, :y]) ) * 100, digits=1)
        end
    end 
end

In [ ]:
replaceMissingDepth(data)

In [ ]:
replaceExtremeValues(data, :depth)

In [ ]:
describe(data)

 À présent, on doit traiter les valeurs manquantes de la variable explicative $cut$. D'abord, on encode la variable $cut$ qui est qualitative pour pouvoir la traiter. Puis, on va aussi faire une régression. La fonction $predictMissingCut()$ permet de choisir les meilleures variables explicatives de cut pour ensuite créer le meilleur modèle pour prédire les valeurs manquantes de cut.

In [ ]:
function checkScore(df::DataFrame, θ̂ ::Array{Float64})
    score = 0;
    best_threshold = 0;
    cut_pred = round.(θ̂ )
    for i in 1:length(df.cut)
        if (cut_pred[i] == df.cut[i])
            score = score + 1
        end
    end
    
    return score
end

In [ ]:
function findBestFormulaCut(data_frame::DataFrame, variables::Vector{Symbol})
    current_variables = Symbol[]
    best_formula, max_area = nothing, 0.0
    variable_count = length(variables)

    for i in 1:variable_count
        current_max_area = 0.0
        best_current_variable_index, best_current_formula = -1, nothing

        for (j, variable) in enumerate(variables)
            if (length(current_variables) != 0)
                formula = Term(:cut) ~ sum(Term(current_variables[k]) for k in 1:length(current_variables)) + Term(variables[j])
                p =  length(current_variables)
            else
                formula = Term(:cut) ~ Term(variables[j])
                p = 1 
            end
            
            M = lm(formula, data_frame)
            θ̂ = convert(Vector{Float64}, predict(M, data_frame))
            area = checkScore(data_frame, θ̂)

            if area > current_max_area
                current_max_area = area
                best_current_variable_index = j
                best_current_formula = formula
            end
        end

        if current_max_area > max_area
            max_area = current_max_area
            best_formula = best_current_formula
            push!(current_variables, variables[best_current_variable_index])
            deleteat!(variables, best_current_variable_index)
        else
            break
        end
    end

    return best_formula
end

In [ ]:
function replaceMissingCut(data::DataFrame) 
    df_cut = dropmissing(data)
    # Créer les mappings pour chaque variable catégorielle
    cut_mapping = Dict("Fair" => 1, "Good" => 2, "Very Good" => 3, "Premium" => 4, "Ideal" => 5)

    # Appliquer les mappings à chaque colonne
    df_cut[!, :cut] = map(x -> cut_mapping[x], df_cut[!, :cut])

    variables_cut = propertynames(select(df_cut, Not([:ID, :cut])))
    best_formula_cut = findBestFormulaCut(df_cut, variables_cut)
    
    M_cut = lm(best_formula_cut, df_cut)
    
    # Extract the rows with missing values
    missing_rows = ismissing.(data[!, :cut])
    missing_rows_indexes = findall(missing_rows)

    # Predict missing values only for rows with missing values
    predicted_values = predict(M_cut, data[missing_rows_indexes, setdiff(names(data), [:cut])])

    # Replace missing values with predicted values
    predicted_cuts = round.(predicted_values)
    for i in 1:length(missing_rows_indexes)
        cut = predicted_values[i]
        if(cut >= 0 && cut <= 1 )
            data[missing_rows_indexes[i], :cut] = "Fair"
        elseif(cut > 1 && cut <= 2 )
            data[missing_rows_indexes[i], :cut] = "Good"
        elseif(cut >2 && cut <= 3 )
            data[missing_rows_indexes[i], :cut] = "Very Good"
        elseif(cut > 3 && cut <= 4 )
            data[missing_rows_indexes[i], :cut] = "Premium"
        else
            data[missing_rows_indexes[i], :cut] = "Ideal"
        end 
    end 
end

In [ ]:
replaceMissingCut(data)

In [ ]:
describe(data)

Suite à la prédiction de cut, on peut voir qu'on a plus de valeurs manquantes.

### Standardisation des données

Afin de traiter plus facilement nos données, on veut pouvoir les standardiser. Ceci permet de mettre toutes nos variables explicatives sur la même échelle.

In [ ]:
function standardize_data(data::DataFrame, cols_standarize::Vector{String})
    # Column names
    col_names = names(data)

    # Convert column names to indices
    name_to_index = Dict(zip(col_names, 1:length(col_names)))

    # Select columns to standardize by name
    data_to_standarize = data[:, [name_to_index[col] for col in cols_standarize]]

    # Fit a Z-score transformation to the selected columns
    dt = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(data_to_standarize), dims=1)
    
    # Transform the selected columns using the Z-score transformation
    standard_data = StatsBase.transform(dt, Matrix{Float64}(data_to_standarize))
    
    return standard_data
end

In [ ]:
cols_standarize = ["x", "y", "z", "depth", "table"]
standard_data = standardize_data(data, cols_standarize)

## Traitement des données catégoriques

La variable cut est la seule qui possède des données manquantes. Nous avons décidé de les remplacer par des données qui possèdent à peu près les mêmes caractéristiques.

Ici, on remarque comme deux courbes. Le volume est donc une bonne variable discriminante. Ceci nous conforte dans notre choix de garder cette nouvelle variable explicative, elle nous sera utile pour construire notre modèle.

## Ajout des colonnes

On pense qu'une nouvelle variable explicative, à savoir le volume du diamant, serait pertinente. Pour cela, on doit d'abord calculer le carré de chacune des dimensions $x$, $y$ et $z$.

### x^2, y^2, z^2, x^3, y^3, z^3, 

In [ ]:
data[:,cols_standarize] = standard_data
data[:, :x²] = (data.x).^2
data[:, :y²] = (data.y).^2
data[:, :z²] = (data.z).^2
data[:, :x³] = (data.x).^3
data[:, :y³] = (data.y).^3
data[:, :z³] = (data.z).^3

### volume

In [ ]:
function diamond_volume(x, y, z, depth, table)
    volume = (x * y * z * (1 - depth / 100) * (1 - table / 100)) 
    return volume
end


In [ ]:
data.volume = diamond_volume.(data.x, data.y, data.z, data.depth, data.table)
data.volume² = (data.volume).^2
data.volume³ = (data.volume).^3

In [ ]:
Gadfly.set_default_plot_size(20cm, 15cm)
Gadfly.plot(data, x="volume", y="price", Geom.point,  
    Guide.xlabel("Volume"), 
    Guide.ylabel("Price"), 
    Guide.title("Diamond Price vs. Volume")
)

## Estimation des paramètres

### Partitionnement des données en ensemble d'entraînement et de validation

In [ ]:
Random.seed!(3302)
train_id = sample(1:nrow(data), round(Int, .8*nrow(data)), ordered=true, replace=false)
valid_id = setdiff(1:nrow(data), train_id)

train = data[train_id,:]
valid = data[valid_id,:]
describe(valid)

### Entrainnement du modèle

In [ ]:
function findBestThreshold(df::DataFrame, ŷ::Vector{Union{Missing, Float64}}, p::Int64)
    y = df.price
    ȳ = mean(y)
    SST = sum((y .- ȳ).^2)
    n = length(y)
    e = y-ŷ
    SSE = e'*e
    R²aj =  1 - SSE/SST * (n-1)/(n-p)
    
    return R²aj
end

In [ ]:
function forwardStepwiseRegressionThreshold(data_frame::DataFrame, variables::Vector{Symbol})
    current_variables = Symbol[]
    max_area = 0.0
    best_formula = nothing

    for i in 1:length(variables)
        current_max_area = 0.0
        best_current_variable_index = -1
        best_current_formula = nothing
        
        for j in 1:length(variables)
            
            if (length(current_variables) != 0)
                formula = Term(:price) ~ sum(Term(current_variables[k]) for k in 1:length(current_variables)) + Term(variables[j])
                p =  length(current_variables)
            else
                formula = Term(:price) ~ Term(variables[j])
                p = 1 
            end
            
            M = lm(formula, data_frame)
            θ̂ = predict(M, data_frame)
            area = findBestThreshold(data_frame, θ̂, p)
            
            if area > current_max_area
                current_max_area = area
                best_current_variable_index = j
                best_current_formula = formula
            end
        end
        
        if current_max_area > max_area
            max_area = current_max_area
            best_formula = best_current_formula
            push!(current_variables, variables[best_current_variable_index])
            deleteat!(variables, best_current_variable_index)
        else
            break
        end
    end
    
    return best_formula
end


In [ ]:
function train_reg_model(dataframe::DataFrame)    
    variables = propertynames(select(dataframe, Not([:ID, :price])))
    best_formula = forwardStepwiseRegressionThreshold(dataframe, variables)
    return best_formula
end

TODO: Decrire la focntion best_formula et interpréter la formule

In [ ]:
best_formula = train_reg_model(data)
println("Meilleur variables ", best_formula)

### Sélection du meilleur modèle

In [ ]:
M = lm(best_formula, data)

## Prédiction à partir des paramètres

### Prédictions locales - ensemble de validation


#### Prédictions locales - ensemble de validation


##### Modèles d'essaies


In [ ]:
function evaluate_model(M, valid::DataFrame, θ̂_valid)
    # Get the number of predictor variables used in the model
    p = length(names(valid)) - 1
    
    y = valid.price
    e = θ̂_valid - y
    
    # Calculate the total sum of squares (SST)
    ȳ = mean(y)
    SST = sum((y .- ȳ).^2)
    
    # Calculate the residual sum of squares (SSE)
    SSE = sum(e .^ 2)
    
    # Calculate the adjusted R-squared value
    n = length(y)
    R²aj = 1 - SSE / SST * (n - 1) / (n - p)
    
    # Calculate the mean squared error (MSE)
    MSE = sqrt(mean(e .^ 2))
    
    return (R²aj = R²aj, MSE = MSE)
end

In [ ]:
θ̂_valid = abs.(predict(M, valid))

result = evaluate_model(M, valid, θ̂_valid)
println("Adjusted R-squared: ", result.R²aj)
println("MSE: ", result.MSE)

In [ ]:
prediction = DataFrame(ID = valid.ID, price_real = valid.price , price_pred = θ̂_valid)
prediction.Ecart = abs.(prediction.price_real - prediction.price_pred)
show(prediction)

In [ ]:
pred_neg = filter(row -> row[:price_pred] <= 0 , prediction)

### Price prédit négativement à analyser.

In [ ]:
values_pred_neg = innerjoin(data,pred_neg , on=:ID)

### Prédictions kaggle - ensemble de test

In [ ]:
data_test_filename = joinpath(data_dir, "test.csv")
test = CSV.read(data_test_filename, DataFrame)
describe(test)

In [ ]:
test = replaceExtremeValues(test, :x)

In [ ]:
test_clone = test[.!ismissing.(test.y), :]
replaceExtremeValues(test_clone, :y)

In [ ]:
predictMissingY(test_clone, test)
replaceExtremeValues(test, :y)

In [ ]:
replaceExtremeValues(test, :z)

In [ ]:
replaceExtremeValues(test, :table)

In [ ]:
describe(test)

In [ ]:
replaceMissingDepth(test)

In [ ]:
describe(test)

In [ ]:
replaceExtremeValues(test, :depth)

In [ ]:
replaceMissingCut(test)

describe(test)

In [ ]:
standard_data = standardize_data(test, cols_standarize)

test[:,cols_standarize] = standard_data
test[:, :x²] = (test.x).^2
test[:, :y²] = (test.y).^2
test[:, :z²] = (test.z).^2
test[:, :x³] = (test.x).^3
test[:, :y³] = (test.y).^3
test[:, :z³] = (test.z).^3

test.volume = diamond_volume.(test.x, test.y, test.z, test.depth, test.table) 
test.volume² = (test.volume).^2
test.volume³ = (test.volume).^3

In [ ]:
describe(test)

In [ ]:
θ̂ = abs.(predict(M, test))
prediction = DataFrame(ID = test.ID, price = θ̂ )

In [ ]:
first(prediction,20)
CSV.write("../data/benchmark_predictions.csv", prediction)

## Conclusion et améliorations

#### Pistes avortées


#### Conclusion 


#### Améliorations